In [5]:
import xml.etree.ElementTree as ET
import json
from random_selector import RandomDocBodyGenerator
import os
from random import randint

tree = ET.parse('../db-final.xml')   
children = list(tree.getroot())  
LIMIT = 2000
allowed_tags = ['author', 'title', 'journal', 'ee', 'publisher', 'year', 'month', 'volume', 'pages', 'booktitle']
paper_titles = {}

i = 0
dict_arr = []                                
for c in children:
    next_paper = False
    l = list(c)                         
    obj = {}
    for element in l:
        if element.tag == 'author':
          if not 'authors' in obj:
            obj['authors'] = []
          obj['authors'].append(element.text)
        else:
          if element.tag in allowed_tags:
            obj[element.tag] = element.text  
            if element.tag == 'title':
              if element.text in paper_titles:
                next_paper = True
              else:
                paper_titles[element.text] = True  

    if next_paper:
      continue          
    obj['type'] = c.tag
    dict_arr.append(obj)
    #print(f"Element #{i} parsed")
    i += 1
    if i == LIMIT:
      break

with open('extracted-features-from-xml.json', 'w') as outfile:
    json.dump(dict_arr, outfile)
    print("Features extraction from XML done")

extracted_data = json.load(open('extracted-features-from-xml.json'))
random_generator = RandomDocBodyGenerator()  
keywords = random_generator.generate_random_keywords()
authors_data = {}
for paper in extracted_data:
    paper["abstract"] = random_generator.generate_random_text()
    paper["documentBody"] = random_generator.generate_random_document_body()
    paper["citations"] = random_generator.get_random_paper_citations(extracted_data)
    paper["keywords"] = random_generator.get_keywords(keywords)
    try:
      paper["language"] = random_generator.get_lang()
    except:
      pass

    if "authors" in paper:
        authors = []
        for author in paper["authors"]:
            if author in authors_data:
                authors.append(authors_data[author])
            else:
                email = author.replace(" ", "_") + "@mail.com"
                bio = random_generator.get_random_sentence()
                affiliation = random_generator.get_affiliation()
                name = author
                birth_year = randint(1950, 2000)
                author = {
                    "name": name,
                    "email": email.lower(),
                    "bio": bio,
                    "affiliations": affiliation,
                    "birth_year" : birth_year
                }
                authors_data[name] = author
                authors.append(author)
        paper["authors"] = authors

        paper["year"] = randint(20 + max(map(lambda author: author["birth_year"], paper["authors"])),2022)

with open("mongodb-documents.json","w") as f:
    json.dump(extracted_data, f)
    print("Random papers data generated")

os.remove("extracted-features-from-xml.json")


Features extraction from XML done
Random papers data generated
